In [1]:
import importlib
from rl import multi_stock_env
importlib.reload(multi_stock_env)
MultiStockEnv = multi_stock_env.MultiStockEnv

In [2]:
from pprint import pprint
from pathlib import Path
import pandas as pd

In [3]:
MARKET = "csi300"
BENCHMARK = "SH000300"

EXP_NAME = "tutorial_exp"

data_provide_uri = "~/qlib_data/cn_data"

# Data

In [4]:
import qlib

qlib.init(provider_uri=data_provide_uri, region="cn")

[755594:MainThread](2025-08-21 00:36:48,252) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[755594:MainThread](2025-08-21 00:36:48,653) INFO - qlib.Initialization - [__init__.py:79] - qlib successfully initialized based on client settings.
[755594:MainThread](2025-08-21 00:36:48,654) INFO - qlib.Initialization - [__init__.py:81] - data_path={'__DEFAULT_FREQ': PosixPath('/root/qlib_data/cn_data')}


In [ ]:
import qlib
from qlib.data import D
import pandas as pd
from stable_baselines3 import PPO
  # 你需要自定义这个环境

# 初始化 Qlib
qlib.init(provider_uri=data_provide_uri, region="cn")
# 获取 csi300 成分股数据
instruments = D.list_instruments(D.instruments("csi300"))

results_df = {}
for instrument in list(instruments)[:10]:  # 只获取前10个股票数据，避免过多请求
    # 获取单个股票数据
    df = D.features(
        [instrument],
        ["$open", "$high", "$low", "$close", "$factor", "$adjclose", "$volume", "$change", "$amount"],
        start_time="2020-01-01",
        end_time="2025-08-31",
    )
    
    # 处理数据
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.columns = [col.replace("$", "") for col in df.columns]

    if not df.empty:
        results_df[instrument] = df
    # 对 volume 做 Min-Max 归一化
    df["volume_norm"] = (df["volume"] - df["volume"].min()) / (df["volume"].max() - df["volume"].min())
    # 

#df = D.features(instruments, ["$close", "$open", "$high", "$low", "$volume"], start_time="2020-01-01", end_time="2025-12-31")



[91441:MainThread](2025-08-19 23:26:25,492) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[91441:MainThread](2025-08-19 23:26:25,494) INFO - qlib.Initialization - [__init__.py:79] - qlib successfully initialized based on client settings.
[91441:MainThread](2025-08-19 23:26:25,495) INFO - qlib.Initialization - [__init__.py:81] - data_path={'__DEFAULT_FREQ': PosixPath('/root/qlib_data/cn_data')}


In [5]:
from qlib.data import D
df_native = D.features(
        D.instruments(MARKET),
        ["$open", "$high", "$low", "$close", "$factor", "$adjclose", "$volume", "$change", "$amount"],
        start_time="2008-01-01",
        end_time="2025-08-31",
    )
df_native.tail(10)

$open     $high      $low    $close   $factor  \
instrument datetime                                                       
SZ302132   2025-08-01  7.471598  7.471598  7.283923  7.285693  0.088526   
           2025-08-04  7.260995  7.480542  7.259224  7.473460  0.088527   
           2025-08-05  7.472465  7.613221  7.420235  7.505220  0.088526   
           2025-08-06  7.455738  8.177232  7.452197  7.862961  0.088527   
           2025-08-07  7.773579  7.832892  7.587672  7.640788  0.088527   
           2025-08-08  7.613321  7.665552  7.515941  7.519483  0.088527   
           2025-08-11  7.505260  7.620345  7.498178  7.536245  0.088526   
           2025-08-12  7.524734  7.681426  7.448602  7.579621  0.088526   
           2025-08-13  7.529137  7.632712  7.521169  7.586679  0.088526   
           2025-08-14  7.576179  7.576179  7.436306  7.465520  0.088527   

                        $adjclose      $volume   $change       $amount  
instrument datetime                                                     
SZ302132   2025-08-01  495.500000  1444457.375 -0.027876  1.062855e+06  
           2025-08-04  508.269989  1326121.250  0.025759  9.851958e+05  
           2025-08-05  510.429993  1428743.875  0.004264  1.075837e+06  
           2025-08-06  534.760010  3891991.250  0.047653  3.045948e+06  
           2025-08-07  519.650024  2343532.000 -0.028259  1.793617e+06  
           2025-08-08  511.399994  1350687.750 -0.015873  1.023233e+06  
           2025-08-11  512.539978  1247831.250  0.002237  9.437501e+05  
           2025-08-12  515.489990  1810458.875  0.005756  1.366656e+06  
           2025-08-13  515.969971  1514339.500  0.000934  1.147399e+06  
           2025-08-14  507.730011  1372873.000 -0.015986  1.026918e+06

In [7]:
## TODO
## 需要对齐股价数据，保证每一行是同个时间点
## 用factor还原实际交易的价格
## 训练集和验证集要分开在两个时间段，做回测

In [8]:
handler_kwargs = {
    "start_time": "2008-01-01",
    "end_time": "2025-08-31",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": MARKET,
}
handler_conf = {
    "class": "Alpha158",
    "module_path": "qlib.contrib.data.handler",
    "kwargs": handler_kwargs,
}
from qlib.utils import init_instance_by_config
hd = init_instance_by_config(handler_conf)


[755594:MainThread](2025-08-21 00:37:43,585) INFO - qlib.timer - [log.py:127] - Time cost: 10.908s | Loading data Done
[755594:MainThread](2025-08-21 00:37:44,303) INFO - qlib.timer - [log.py:127] - Time cost: 0.186s | DropnaLabel Done
[755594:MainThread](2025-08-21 00:37:46,169) INFO - qlib.timer - [log.py:127] - Time cost: 1.865s | CSZScoreNorm Done
[755594:MainThread](2025-08-21 00:37:46,171) INFO - qlib.timer - [log.py:127] - Time cost: 2.585s | fit & process data Done
[755594:MainThread](2025-08-21 00:37:46,172) INFO - qlib.timer - [log.py:127] - Time cost: 13.496s | Init data Done


In [14]:
df = hd.fetch()
# 假设 df 是 MultiIndex ['datetime', 'instrument']
all_dates = set(df.index.get_level_values('datetime'))
instruments = df.index.get_level_values('instrument').unique()

valid_stocks = []
for inst in instruments:
    inst_dates = set(df.loc[(slice(None), inst), :].index.get_level_values('datetime'))
    if inst_dates == all_dates:
        valid_stocks.append(inst)
valid_stocks = valid_stocks[:1]  # 只取前5个股票
print(valid_stocks)

['SH600000']


In [15]:
# 假设 valid_stocks 是你筛选出来的股票列表
df_filtered = df[df.index.get_level_values('instrument').isin(valid_stocks)]

df_merged = df_filtered.join(df_native[['$adjclose']], how='left')

In [16]:
df_merged = df_merged[~df_merged['$adjclose'].isna()]

In [17]:
# 假设 df 是 Alpha158 处理后的 DataFrame，索引为 ['datetime', 'instrument']
df_merged

# 划分训练集和验证集
train_mask = (df_merged.index.get_level_values('datetime') >= '2020-01-01') & (df_merged.index.get_level_values('datetime') <= '2023-12-31')
valid_mask = (df_merged.index.get_level_values('datetime') >= '2024-01-01') & (df_merged.index.get_level_values('datetime') <= '2025-08-31')
train_df = df_merged[train_mask]
valid_df = df_merged[valid_mask]

# 转为 dict 格式
def df_to_dict(long_df):
    df_dict = {}
    for inst, subdf in long_df.groupby('instrument'):
        subdf = subdf.sort_index(level='datetime')
        subdf = subdf.reset_index()
        df_dict[inst] = subdf
    return df_dict

train_dict = df_to_dict(train_df)
valid_dict = df_to_dict(valid_df)
tickers = list(train_dict.keys())
tickers_valid = list(valid_dict.keys())

# 传入 RL 环境
env_train = MultiStockEnv(train_dict, tickers)
env_valid = MultiStockEnv(valid_dict, tickers_valid)

In [ ]:
from stable_baselines3 import PPO
import torch

torch.set_default_dtype(torch.float32)
# 训练
model = PPO(
    "MlpPolicy",
    env_train,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=1024,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
    verbose=1,
    policy_kwargs=dict(net_arch=[256, 256])
)
model.learn(total_timesteps=300000,)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/opt/software/miniconda3/envs/quant/lib/python3.13/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 263      |
|    ep_rew_mean     | -0.186   |
| time/              |          |
|    fps             | 400      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 351           |
|    ep_rew_mean          | -0.235        |
| time/                   |               |
|    fps                  | 429           |
|    iterations           | 2             |
|    time_elapsed         | 9             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00013753932 |
|    clip_fraction        | 0.0063        |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.42         |
|    explained_variance   | -1.04         |


In [ ]:

# 回测
obs, _ = env_valid.reset()
done = False
asset = 1000000
asset_curve = [asset]
while not done:
    action, _states = model.predict(obs)
    #print(action)
    obs, reward, done, _, _ = env_valid.step(action)
    new_asset = 10 ** reward * asset
    asset_curve.append(new_asset)
res = env_valid.evaluate_performance(asset_curve)
print(asset_curve)
print(res)
# 评估策略表现
env_valid.render()

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.13837016]
[1.]
[0.1821073]
[0.81647074]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1000000, np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+06), np.float32(1e+